# Liriscat paper experiments
### 1. Init
#### 1.1. Import libraries (necessary)

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTHONHASHSEED"] = "0"
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"

import liriscat
liriscat.utils.set_seed(0)

import logging
import gc
import json
import torch
liriscat.utils.set_seed(0)
import pandas as pd
from importlib import reload

#### 1.2. Set up the loggers (recommended)

In [2]:
liriscat.utils.setuplogger(verbose = True, log_name="liriscat", debug=False)

### 2. CDM prediction
#### 2.1. Training and testing, sequential version

In [3]:
import warnings

gc.collect()
torch.cuda.empty_cache()

In [ ]:
config = liriscat.utils.generate_eval_config(load_params=True, esc = 'error', valid_metric= 'mi_acc', pred_metrics = ["mi_acc"], profile_metrics = ['meta_doa'], save_params=False, n_query=6, num_epochs=100, batch_size=512)
liriscat.utils.set_seed(config["seed"])

config["dataset_name"] = "math2"
logging.info(config["dataset_name"])
config['learning_rate'] = 0.0001
config['inner_user_lr'] = 0.016848380924625605
config['lambda'] = 9.972254466547545e-06

config['meta_lr'] = 0.5
config['learning_users_emb_lr'] = 0.0005
config['patience'] = 20#5
config['num_inner_users_epochs'] = 3
config['d_in'] = 4
config['num_responses'] = 12

#pred_metrics,df_interp = test(config)

CUDA is available. Using GPU.
[INFO 38:14] math2


In [5]:
logging.info(f'#### {config["dataset_name"]} ####')
logging.info(f'#### config : {config} ####')
config['embs_path']='../embs/'
config['params_path']='../ckpt/'

gc.collect()
torch.cuda.empty_cache()

# Dataset downloading for doa and rm
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Concept map format : {question_id : [category_id1, category_id2, ...]}
concept_map = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

## Metadata map format : {"num_user_id": ..., "num_item_id": ..., "num_dimension_id": ...}
metadata = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_metadata.json', 'r'))


## Tensor containing the nb of modalities per question
nb_modalities = torch.load(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_nb_modalities.pkl',weights_only=True)


[INFO 38:14] #### math2 ####


[INFO 38:14] #### config : {'seed': 0, 'dataset_name': 'math2', 'load_params': True, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'mi_acc', 'learning_rate': 0.0001, 'batch_size': 512, 'valid_batch_size': 10000, 'num_epochs': 100, 'eval_freq': 1, 'patience': 20, 'device': device(type='cuda'), 'lambda': 9.972254466547545e-06, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['mi_acc'], 'profile_metrics': ['meta_doa'], 'num_responses': 12, 'low_mem': False, 'n_query': 6, 'CDM': 'impact', 'i_fold': 0, 'num_inner_users_epochs': 3, 'num_inner_epochs': 10, 'inner_lr': 0.0001, 'inner_user_lr': 0.016848380924625605, 'meta_lr': 0.5, 'meta_trainer': 'Adam', 'num_workers': 0, 'pin_memory': False, 'debug': False, 'learning_users_emb_lr': 0.001, 'd_in': 4} ####


In [6]:
meta_trainers = ['GAP']
for meta_trainer in meta_trainers : 
    config['meta_trainer'] = meta_trainer
    logging.info(f'#### meta_trainer : {config["meta_trainer"]} ####')
    for i_fold in range(1,2) : 
        config['i_fold'] = i_fold
            
        logging.info(f'#### i_fold : {i_fold} ####')
        ## Dataframe columns : (user_id, question_id, response, category_id)
        train_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        valid_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        test_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})

        train_data = liriscat.dataset.CATDataset(train_df, concept_map, metadata, config,nb_modalities)
        valid_data = liriscat.dataset.EvalDataset(valid_df, concept_map, metadata, config,nb_modalities)
        test_data = liriscat.dataset.EvalDataset(test_df, concept_map, metadata, config,nb_modalities)

        for seed in range(1) :
            config['seed'] = seed
            logging.info(f'#### seed : {seed} ####')

            train_data.reset_rng()
            valid_data.reset_rng()
            test_data.reset_rng()

            S = liriscat.selectionStrategy.Random(metadata,**config)
            S.init_models(train_data, valid_data)
            S.train(train_data, valid_data)
            liriscat.utils.set_seed(0)
            S.reset_rng()
            d = (S.evaluate_test(test_data, train_data, valid_data))
            logging.info(d)
            logging.info(liriscat.utils.pareto_index(d))

    torch.cuda.empty_cache()

[INFO 38:14] #### meta_trainer : GAP ####
[INFO 38:14] #### i_fold : 1 ####
[INFO 38:30] #### seed : 0 ####
[INFO 38:30] Random_cont_model
[INFO 38:30] compiling CDM model
[INFO 38:32] compiling selection model
[INFO 38:32] ------- START Training
[INFO 38:32] train on cuda


/home/arthur/liriscat/liriscat/selectionStrategy/random.py:30: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
  0%|          | 0/100 [00:00<?, ?it/s]

[INFO 38:33] ----- User batch : 0
[INFO 38:39] ----- User batch : 1
[INFO 38:43] ----- User batch : 2
[INFO 38:48] ----- User batch : 3
[INFO 38:52] ----- User batch : 4
[INFO 38:55] - meta_params gn: [0.031223371624946594, 0.035650260746479034]
[INFO 38:55] - cross_cond gn: [0.06907379627227783, 0.16024543344974518]
[INFO 38:55] - meta_lambda gn: 0.957507312297821
[INFO 38:55] - learning_users_emb gn: 26.643220901489258
[INFO 38:55] - meta_params norm: [34.032470703125, 80.113525390625]
[INFO 38:55] - cross_cond norm: [33.01451110839844, 80.7965087890625]
[INFO 38:55] - meta_lambda norm: 11.515698432922363
[INFO 38:55] - learning_users_emb norm: 0.12323136627674103
[INFO 38:55] - meta_params: 0.5
[INFO 38:55] - cross_cond: 0.5
[INFO 38:55] - meta_lambda: 0.5
[INFO 38:55] - learning_users_emb: 0.001
[INFO 39:03] rmse : 0.5412712693214417
[INFO 39:03] valid_rmse : 0.6921627521514893
[INFO 39:03] valid_loss : 0.937843382358551


  1%|          | 1/100 [00:29<49:00, 29.70s/it]

[INFO 39:03] ----- User batch : 0
[INFO 39:08] ----- User batch : 1
[INFO 39:13] ----- User batch : 2
[INFO 39:17] ----- User batch : 3
[INFO 39:21] ----- User batch : 4
[INFO 39:24] - meta_params gn: [0.024786455556750298, 0.03819642588496208]
[INFO 39:24] - cross_cond gn: [0.034133296459913254, 0.0841982439160347]
[INFO 39:24] - meta_lambda gn: 0.5134706497192383
[INFO 39:24] - learning_users_emb gn: 27.01006507873535
[INFO 39:24] - meta_params norm: [35.243019104003906, 80.23528289794922]
[INFO 39:24] - cross_cond norm: [32.145660400390625, 80.67011260986328]
[INFO 39:24] - meta_lambda norm: 11.016040802001953
[INFO 39:24] - learning_users_emb norm: 0.12008725106716156
[INFO 39:24] - meta_params: 0.5
[INFO 39:24] - cross_cond: 0.5
[INFO 39:24] - meta_lambda: 0.5
[INFO 39:24] - learning_users_emb: 0.001
[INFO 39:31] rmse : 0.5479499101638794
[INFO 39:31] valid_rmse : 0.7007032036781311
[INFO 39:31] valid_loss : 0.9383835792541504


...

[INFO 48:34] ----- User batch : 0
[INFO 48:37] ----- User batch : 1
[INFO 48:40] ----- User batch : 2
[INFO 48:44] ----- User batch : 3
[INFO 48:47] ----- User batch : 4
[INFO 48:49] - meta_params gn: [0.017445309087634087, 0.016455091536045074]
[INFO 48:49] - cross_cond gn: [0.022621063515543938, 0.05311064422130585]
[INFO 48:49] - meta_lambda gn: 0.08843137323856354
[INFO 48:49] - learning_users_emb gn: 27.61710548400879
[INFO 48:49] - meta_params norm: [42.751888275146484, 81.08473205566406]
[INFO 48:49] - cross_cond norm: [30.458105087280273, 80.24561309814453]
[INFO 48:49] - meta_lambda norm: 11.243287086486816
[INFO 48:49] - learning_users_emb norm: 0.09882801026105881
[INFO 48:49] - meta_params: 0.00390625
[INFO 48:49] - cross_cond: 0.00390625
[INFO 48:49] - meta_lambda: 0.00390625
[INFO 48:49] - learning_users_emb: 7.8125e-06
[INFO 48:55] rmse : 0.5442090034484863
[INFO 48:55] valid_rmse : 0.6959194540977478
[INFO 48:55] valid_loss : 0.9379234313964844


 29%|██▉       | 29/100 [10:21<25:21, 21.43s/it]

[INFO 48:55] -- END Training --
[INFO 48:55] train on cuda



100%|██████████| 6/6 [00:04<00:00,  1.32it/s]


[INFO 49:05] ({0: {'mi_acc': 0.6921773552894592}, 1: {'mi_acc': 0.6917787790298462}, 2: {'mi_acc': 0.6956651210784912}, 3: {'mi_acc': 0.6956651210784912}, 4: {'mi_acc': 0.7011459469795227}, 5: {'mi_acc': 0.7012456059455872}}, {0: {'meta_doa': 0.53188928737275}, 1: {'meta_doa': 0.557330235019339}, 2: {'meta_doa': 0.5872005357851973}, 3: {'meta_doa': 0.614925825417725}, 4: {'meta_doa': 0.6429254791597345}, 5: {'meta_doa': 0.6627757848988324}})
[INFO 49:05] 0.11817899263910915


In [ ]:
config['meta_trainer'] = 'GAP'
#config['meta_trainer'] = 'Adam'
for seed in range(1) :
    config['seed'] = seed
    logging.info(f'#### seed : {seed} ####')
    
    train_data.reset_rng()
    valid_data.reset_rng()
    test_data.reset_rng()


    S = liriscat.selectionStrategy.Random(metadata,**config)
    S.init_models(train_data, valid_data)
    S.train(train_data, valid_data)
    liriscat.utils.set_seed(0)
    S.reset_rng()
    d = (S.evaluate_test(test_data, train_data, valid_data))
    print(liriscat.utils.pareto_index(d))
    print(d)
    torch.cuda.empty_cache()

[INFO 47:13] #### seed : 0 ####
[INFO 47:13] Random_cont_model
[INFO 47:13] compiling CDM model
[INFO 47:13] compiling selection model
[INFO 47:13] ------- START Training
[INFO 47:13] train on cuda


/home/arthur/liriscat/liriscat/selectionStrategy/random.py:30: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
  0%|          | 0/100 [00:00<?, ?it/s]

[INFO 47:14] ----- User batch : 0
[INFO 47:19] ----- User batch : 1
[INFO 47:22] ----- User batch : 2
[INFO 47:25] ----- User batch : 3
[INFO 47:28] ----- User batch : 4
[INFO 47:30] - meta_params gn: [0.031223371624946594, 0.035650260746479034]
[INFO 47:30] - cross_cond gn: [0.06907379627227783, 0.16024543344974518]
[INFO 47:30] - meta_lambda gn: 0.957507312297821
[INFO 47:30] - learning_users_emb gn: 26.643220901489258
[INFO 47:30] - meta_params norm: [34.032470703125, 80.113525390625]
[INFO 47:30] - cross_cond norm: [33.01451110839844, 80.7965087890625]
[INFO 47:30] - meta_lambda norm: 11.515698432922363
[INFO 47:30] - learning_users_emb norm: 0.12323136627674103
[INFO 47:30] - meta_params: 0.5
[INFO 47:30] - cross_cond: 0.5
[INFO 47:30] - meta_lambda: 0.5
[INFO 47:30] - learning_users_emb: 0.005
[INFO 47:37] rmse : 0.5411791801452637
[INFO 47:37] valid_rmse : 0.6920449733734131
[INFO 47:37] valid_loss : 0.9378431439399719


  1%|          | 1/100 [00:23<38:18, 23.22s/it]

[INFO 47:37] ----- User batch : 0
[INFO 47:40] ----- User batch : 1
[INFO 47:44] ----- User batch : 2
[INFO 47:47] ----- User batch : 3
[INFO 47:50] ----- User batch : 4
[INFO 47:53] - meta_params gn: [0.021900063380599022, 0.02990972064435482]
[INFO 47:53] - cross_cond gn: [0.03406094014644623, 0.08404725044965744]
[INFO 47:53] - meta_lambda gn: 0.22097423672676086
[INFO 47:53] - learning_users_emb gn: 26.895105361938477
[INFO 47:53] - meta_params norm: [35.243019104003906, 80.23528289794922]
[INFO 47:53] - cross_cond norm: [32.145660400390625, 80.67011260986328]
[INFO 47:53] - meta_lambda norm: 11.016040802001953
[INFO 47:53] - learning_users_emb norm: 0.10807743668556213
[INFO 47:53] - meta_params: 0.5
[INFO 47:53] - cross_cond: 0.5
[INFO 47:53] - meta_lambda: 0.5
[INFO 47:53] - learning_users_emb: 0.005
[INFO 47:57] rmse : 0.5478590130805969
[INFO 47:57] valid_rmse : 0.7005869746208191
[INFO 47:57] valid_loss : 0.9383183121681213


...

[INFO 57:13] ----- User batch : 0
[INFO 57:16] ----- User batch : 1
[INFO 57:19] ----- User batch : 2
[INFO 57:22] ----- User batch : 3
[INFO 57:25] ----- User batch : 4
[INFO 57:27] - meta_params gn: [0.02700931951403618, 0.029903776943683624]
[INFO 57:27] - cross_cond gn: [0.03397426754236221, 0.09785724431276321]
[INFO 57:27] - meta_lambda gn: 0.9443564414978027
[INFO 57:27] - learning_users_emb gn: 23.33382797241211
[INFO 57:27] - meta_params norm: [37.512001037597656, 80.44165802001953]
[INFO 57:27] - cross_cond norm: [29.116575241088867, 80.0670394897461]
[INFO 57:27] - meta_lambda norm: 12.322739601135254
[INFO 57:27] - learning_users_emb norm: 0.06543269008398056
[INFO 57:27] - meta_params: 0.00390625
[INFO 57:27] - cross_cond: 0.00390625
[INFO 57:27] - meta_lambda: 0.00390625
[INFO 57:27] - learning_users_emb: 3.90625e-05
[INFO 57:31] rmse : 0.5490400195121765
[INFO 57:31] valid_rmse : 0.7020971775054932
[INFO 57:31] valid_loss : 0.937699019908905


 29%|██▉       | 29/100 [10:17<25:12, 21.31s/it]

[INFO 57:31] -- END Training --
[INFO 57:31] train on cuda



100%|██████████| 6/6 [00:04<00:00,  1.49it/s]


0.10589458560051394
({0: {'mi_acc': 0.6842052340507507}, 1: {'mi_acc': 0.6829097867012024}, 2: {'mi_acc': 0.6902839541435242}, 3: {'mi_acc': 0.6916791200637817}, 4: {'mi_acc': 0.6974588632583618}, 5: {'mi_acc': 0.7022421360015869}}, {0: {'meta_doa': 0.5295431083496138}, 1: {'meta_doa': 0.5608702564859536}, 2: {'meta_doa': 0.5825789078715902}, 3: {'meta_doa': 0.6021081697438648}, 4: {'meta_doa': 0.6261025073719617}, 5: {'meta_doa': 0.6440516914619652}})
